In [1]:
import findspark
findspark.init()
import pyspark
sc = pyspark.SparkContext(appName="Spark2")

In [2]:
# read in genre and artist
source_rdd = sc.textFile("artists_with_genres.csv",100)

In [3]:
# see what we got
source_rdd.take(5)

[u',artist,name,genre',
 u'0,03098741-08b3-4dd7-b3f6-1b0bfa2c879c,Liars,alternative rock',
 u'1,69c4cc43-8163-41c5-ac81-30946d27bb69,CunninLynguists,american',
 u'2,7a2e6b55-f149-4e74-be6a-30a1b1a387bb,The Desert Sessions,rock and indie',
 u'3,7002bf88-1269-4965-a772-4ba1e7a91eaa,Glenn Gould,canadian pianist']

In [4]:
# remove header
header = source_rdd.first()
source_no_head_rdd = source_rdd.filter(lambda x: x != header)

In [48]:
# Compute how many artists there are per genre and how many different genres there are
artists_by_genre = source_no_head_rdd.map(lambda x: x.split(','))\
                            .map(lambda (row_id,artist_id,artist,genre): (genre, 1))\
                            .reduceByKey(lambda a,b: a+b)

In [50]:
print artists_by_genre.count()
artists_by_genre.take(5)

354


[(u'', 206),
 (u'heartland rock', 3),
 (u'norway', 1),
 (u'neo-psychedelia', 2),
 (u'new wave', 11)]

In [5]:
# create rdd where key = user, value = list of (genre, avg # of plays per artist of that genre)
# read in train
source_train_rdd = sc.textFile("train.csv",100)

In [6]:
# remove header
train_header = source_train_rdd.first()
source_train_no_head_rdd = source_train_rdd.filter(lambda x: x != train_header)

In [7]:
# See how many different artists each user has listened to
user_artists_rdd = source_train_no_head_rdd.map(lambda x: x.split(','))\
                           .map(lambda (user_id, artist_id,plays): (user_id,1))\
                           .reduceByKey(lambda a,b: a+b)

In [17]:
user_artists_rdd.take(5)

[(u'b495533170ac9a9dd20de8a6dd1d13d87afe2929', 18),
 (u'98397a9c9caf9d767fb4e839a95666ef7b72ef21', 9),
 (u'3f4166a1112d90b2e2dbc7ae404ceede5edafbfe', 23),
 (u'4793b73622a902b0f2649240b99f19844dc1f1f1', 15),
 (u'c975eb6595be16834f408af60925be5603d1f3e6', 21)]

In [21]:
# Get how many total listens each user has
user_counts_rdd = source_train_no_head_rdd.map(lambda x: x.split(','))\
                                          .map(lambda (user_id,artist_id,plays): (user_id,plays))\
                                          .reduceByKey(lambda a,b: int(a)+int(b))

In [22]:
user_counts_rdd.take(5)

[(u'b495533170ac9a9dd20de8a6dd1d13d87afe2929', 10752),
 (u'98397a9c9caf9d767fb4e839a95666ef7b72ef21', 1589),
 (u'3f4166a1112d90b2e2dbc7ae404ceede5edafbfe', 12931),
 (u'4793b73622a902b0f2649240b99f19844dc1f1f1', 229),
 (u'c975eb6595be16834f408af60925be5603d1f3e6', 214)]

In [39]:
# Compute the average number of listens each user gives to each artist
user_avglistens_rdd = user_counts_rdd.join(user_artists_rdd).mapValues(lambda v: v[0]/float(v[1]))

In [40]:
user_avglistens_rdd.take(5)

[(u'b495533170ac9a9dd20de8a6dd1d13d87afe2929', 597.3333333333334),
 (u'98397a9c9caf9d767fb4e839a95666ef7b72ef21', 176.55555555555554),
 (u'3f4166a1112d90b2e2dbc7ae404ceede5edafbfe', 562.2173913043479),
 (u'4793b73622a902b0f2649240b99f19844dc1f1f1', 15.266666666666667),
 (u'c975eb6595be16834f408af60925be5603d1f3e6', 10.19047619047619)]